# LLM Optimization Modelling Experiment

In [30]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel
from IPython.display import Markdown

## 1. Define the problem description

In [31]:
problem = '''We are delighted to welcome you, our newest intern on the Analytics team of Massachusetts General Hospital! You have been placed in a challenging role where you will be tasked with solving a real-world problem in the field of medical physics. We are building a pilot program in Boston, and if successful, your work could be applied widely in hospitals with limited capacity in many countries.

You are responsible for determining the best treatment plan for 17 patients who require radiotherapy. Your goal is to optimize the use of two possible treatments: photon therapy and proton therapy. While proton therapy is known to target tumors more precisely, it is also more expensive and has limited capacity in many countries. Therefore, you will need to balance the benefits of proton therapy with its limitations and cost to create an effective treatment plan for each patient.

To determine the best course of action for each patient, you will use a scoring system called the Biological Equivalent Dose (BED). This system allows you to calculate the effectiveness of each patient’s treatment plan by considering the number of proton fractions that can be used while still achieving the highest possible BED.

We have n=17 patients who need radiotherapy. Each patient i needs 15 fractions, which can be photon fractions, proton fractions, or a mix of photon and proton fractions (e.g. 4 proton fractions and 11 photon fractions). We want to use the limited proton therapy capacity as best as possible. We can calculate the BED score for each patient when p proton fractions and 15-p photon fractions are used, as BEDi(p,15-p), i.e., the BED when p proton and 15-p photon fractions are delivered for patient i. The higher the score, the better. 

The data file "ProblemData.csv" contains a 2D matrix of BED scores. It does not have an index. It was made in Excel and saved as csv. The columns are the number of proton fractions and each row represents a patient. In particular, the number at the (i,j) position is the score for patient i receiving j proton fractions. 

Suppose that the total maximal capacity C is 100 proton fractions. To maximize the total BED scores for all the patients, which patients should get proton fractions, and how many should they get? Formulate an integer linear optimization model to solve this problem. Assume you know the value BEDi(j,15-j) for each patient i. 
'''

## 2. Generate the mathematical model

In [32]:
#Initializing the session. To replicate, make sure the right credentials are saved in a PATH variable
PROJECT_ID = "llm4optproblems"
REGION = "us-central1"
vertexai.init(project=PROJECT_ID, location=REGION)

#Specifying the model
generative_multimodal_model = GenerativeModel("gemini-1.5-pro-preview-0409")

#The propmt applied to all problems
prompt = '''Let's think step by step. Please write a mathematical optimization model for this problem. If there are parameter values, make sure to include them in the mathematical formulation.
'''

#Generate the response
response = generative_multimodal_model.generate_content([prompt+problem])


In [33]:
#Show the resopnse in a formatted way
Markdown(response.text)

## Integer Linear Optimization Model for Radiotherapy Treatment Planning

### Sets and Indices:

*  $I$: Set of patients, indexed by $i$ (where $|I| = n = 17$)
*  $P$: Set of possible proton fractions, indexed by $p$ (where $P = \{0, 1, 2, ..., 15\}$)

### Parameters:

*  $BED_{i,p}$: Biological Equivalent Dose score for patient $i$ receiving $p$ proton fractions
*  $C$: Total capacity of proton fractions (given as 100)

### Decision Variables:

*  $x_{i,p} \in \{0, 1\}$: Binary variable that equals 1 if patient $i$ receives $p$ proton fractions, and 0 otherwise. 

### Objective Function:

Maximize the total BED score for all patients:

```
Maximize Z = ∑_{i∈I} ∑_{p∈P} BED_{i,p} * x_{i,p}
```

### Constraints:

1. **Each patient receives exactly 15 fractions:**
    *  For each patient $i$:
    ```
    ∑_{p∈P} x_{i,p} = 1 
    ```
2. **Total proton fractions used must not exceed capacity:**
    ```
    ∑_{i∈I} ∑_{p∈P} p * x_{i,p} <= C
    ```

### Model Summary:

This integer linear optimization model aims to maximize the total BED score for all patients while adhering to the constraints of limited proton therapy capacity and ensuring each patient receives the required number of fractions. 

### Implementation:

This model can be implemented and solved using various optimization software packages such as CPLEX, Gurobi, or PuLP (Python). By inputting the BED scores from the "ProblemData.csv" file and setting the capacity constraint, the model will determine the optimal allocation of proton fractions to patients, maximizing the overall treatment effectiveness.


## 3. Generate the pyomo code

In [34]:
#Second prompt gets the output of the previous step and generates the code
prompt2 = "Please write pyomo code for this mathematical problem. Use sample data where needed. Indicate where you use sample data. \n"
prompt2 += response.text 
response2 = generative_multimodal_model.generate_content([prompt2])

In [35]:
#Showing the code in a formatted way
Markdown(response2.text)

## Pyomo Implementation of Radiotherapy Treatment Planning Model

```python
from pyomo.environ import *

# Sample Data (replace with actual data from ProblemData.csv)
BED = {
    (1, 0): 60, (1, 1): 65,  ..., (1, 15): 100,
    (2, 0): 55, (2, 1): 62,  ..., (2, 15): 95,
    ...
    (17, 0): 70, (17, 1): 78, ..., (17, 15): 110 
}
C = 100  # Total capacity of proton fractions

# Define the model
model = ConcreteModel()

# Sets and Indices
model.I = Set(initialize=range(1, 18))  # Set of patients (1 to 17)
model.P = Set(initialize=range(16))    # Set of possible proton fractions (0 to 15)

# Parameters
model.BED = Param(model.I, model.P, initialize=BED)
model.C = Param(initialize=C)

# Decision Variables
model.x = Var(model.I, model.P, domain=Binary)

# Objective Function
def total_BED_rule(model):
    return sum(model.BED[i, p] * model.x[i, p] for i in model.I for p in model.P)
model.total_BED = Objective(rule=total_BED_rule, sense=maximize)

# Constraints
def one_fraction_per_patient_rule(model, i):
    return sum(model.x[i, p] for p in model.P) == 1
model.one_fraction_constraint = Constraint(model.I, rule=one_fraction_per_patient_rule)

def capacity_constraint_rule(model):
    return sum(p * model.x[i, p] for i in model.I for p in model.P) <= model.C
model.capacity_constraint = Constraint(rule=capacity_constraint_rule)

# Solve the model (requires a solver like CPLEX or Gurobi)
solver = SolverFactory('cplex')  # Replace with your chosen solver
solver.solve(model)

# Print the results
for i in model.I:
    for p in model.P:
        if model.x[i, p].value == 1:
            print(f"Patient {i} receives {p} proton fractions.")

print(f"Total BED score: {model.total_BED()}") 
```

**Explanation:**

1. **Sample Data:** The `BED` dictionary is where you would replace the sample data with the actual BED scores from your "ProblemData.csv" file.
2. **Sets and Indices:**  We define sets for patients (`model.I`) and proton fractions (`model.P`). 
3. **Parameters:** We define the `BED` parameter using the provided sample data and the `C` parameter for capacity.
4. **Decision Variables:** The `model.x` variable is defined as a binary variable representing the treatment plan. 
5. **Objective Function:** The `total_BED_rule` defines the objective of maximizing the total BED score.
6. **Constraints:** 
    * `one_fraction_per_patient_rule` ensures each patient receives exactly one type of fraction treatment.
    * `capacity_constraint_rule` ensures the total used proton fractions do not exceed the capacity. 
7. **Solving:** The model is solved using a solver like CPLEX or Gurobi. You would need to install and configure the solver separately. 
8. **Results:** The code prints the optimal treatment plan (number of fractions per patient) and the total BED score. 


## 4. Input problem data and try running the generated code

In [44]:
from pyomo.environ import *

# # Sample Data (replace with actual data from ProblemData.csv)
# BED = {
#     (1, 0): 60, (1, 1): 65,  ..., (1, 15): 100,
#     (2, 0): 55, (2, 1): 62,  ..., (2, 15): 95,
#     ...
#     (17, 0): 70, (17, 1): 78, ..., (17, 15): 110 
# }

#Data input done by human
df = pd.read_csv('ProblemData.csv', header=None)
BED = {}
for i, row in df.iterrows():
    for j, dose in enumerate(row):
        BED[(i+1, j)] = dose
#End
        
C = 100  # Total capacity of proton fractions

# Define the model
model = ConcreteModel()

# Sets and Indices
model.I = Set(initialize=range(1, 18))  # Set of patients (1 to 17)
model.P = Set(initialize=range(16))    # Set of possible proton fractions (0 to 15)

# Parameters
model.BED = Param(model.I, model.P, initialize=BED)
model.C = Param(initialize=C)

# Decision Variables
model.x = Var(model.I, model.P, domain=Binary)

# Objective Function
def total_BED_rule(model):
    return sum(model.BED[i, p] * model.x[i, p] for i in model.I for p in model.P)
model.total_BED = Objective(rule=total_BED_rule, sense=maximize)

# Constraints
def one_fraction_per_patient_rule(model, i):
    return sum(model.x[i, p] for p in model.P) == 1
model.one_fraction_constraint = Constraint(model.I, rule=one_fraction_per_patient_rule)

def capacity_constraint_rule(model):
    return sum(p * model.x[i, p] for i in model.I for p in model.P) <= model.C
model.capacity_constraint = Constraint(rule=capacity_constraint_rule)

# Solve the model (requires a solver like CPLEX or Gurobi)
solver = SolverFactory('glpk')  # Replace with your chosen solver
solver.solve(model)

# Print the results
for i in model.I:
    for p in model.P:
        if model.x[i, p].value == 1:
            print(f"Patient {i} receives {p} proton fractions.")

print(f"Total BED score: {model.total_BED()}")

Patient 1 receives 8 proton fractions.
Patient 2 receives 8 proton fractions.
Patient 3 receives 3 proton fractions.
Patient 4 receives 0 proton fractions.
Patient 5 receives 5 proton fractions.
Patient 6 receives 0 proton fractions.
Patient 7 receives 4 proton fractions.
Patient 8 receives 15 proton fractions.
Patient 9 receives 4 proton fractions.
Patient 10 receives 5 proton fractions.
Patient 11 receives 6 proton fractions.
Patient 12 receives 0 proton fractions.
Patient 13 receives 10 proton fractions.
Patient 14 receives 0 proton fractions.
Patient 15 receives 10 proton fractions.
Patient 16 receives 10 proton fractions.
Patient 17 receives 12 proton fractions.
Total BED score: 8.239999999999998


## 5. Correct the code to verify model viability (optional)

## 6. Printing the outputs as strings, so they can be saved.
Those can be rendered as markdown for better readability

In [46]:
print(response.text)

## Integer Linear Optimization Model for Radiotherapy Treatment Planning

### Sets and Indices:

*  $I$: Set of patients, indexed by $i$ (where $|I| = n = 17$)
*  $P$: Set of possible proton fractions, indexed by $p$ (where $P = \{0, 1, 2, ..., 15\}$)

### Parameters:

*  $BED_{i,p}$: Biological Equivalent Dose score for patient $i$ receiving $p$ proton fractions
*  $C$: Total capacity of proton fractions (given as 100)

### Decision Variables:

*  $x_{i,p} \in \{0, 1\}$: Binary variable that equals 1 if patient $i$ receives $p$ proton fractions, and 0 otherwise. 

### Objective Function:

Maximize the total BED score for all patients:

```
Maximize Z = ∑_{i∈I} ∑_{p∈P} BED_{i,p} * x_{i,p}
```

### Constraints:

1. **Each patient receives exactly 15 fractions:**
    *  For each patient $i$:
    ```
    ∑_{p∈P} x_{i,p} = 1 
    ```
2. **Total proton fractions used must not exceed capacity:**
    ```
    ∑_{i∈I} ∑_{p∈P} p * x_{i,p} <= C
    ```

### Model Summary:

This integer linear opt

In [47]:
print(response2.text)

## Pyomo Implementation of Radiotherapy Treatment Planning Model

```python
from pyomo.environ import *

# Sample Data (replace with actual data from ProblemData.csv)
BED = {
    (1, 0): 60, (1, 1): 65,  ..., (1, 15): 100,
    (2, 0): 55, (2, 1): 62,  ..., (2, 15): 95,
    ...
    (17, 0): 70, (17, 1): 78, ..., (17, 15): 110 
}
C = 100  # Total capacity of proton fractions

# Define the model
model = ConcreteModel()

# Sets and Indices
model.I = Set(initialize=range(1, 18))  # Set of patients (1 to 17)
model.P = Set(initialize=range(16))    # Set of possible proton fractions (0 to 15)

# Parameters
model.BED = Param(model.I, model.P, initialize=BED)
model.C = Param(initialize=C)

# Decision Variables
model.x = Var(model.I, model.P, domain=Binary)

# Objective Function
def total_BED_rule(model):
    return sum(model.BED[i, p] * model.x[i, p] for i in model.I for p in model.P)
model.total_BED = Objective(rule=total_BED_rule, sense=maximize)

# Constraints
def one_fraction_per_patient_rul